# Tentamen functioneel programmeren 2021-2022 Kans 1

Het tentamen bestaat uit twee delen:
- 3 Begrips- en programmeervragen (60 punten)
- 1 RxAvansvraag (30 punten)

Bij 45 van de 90 punten heb je een 5,5. 

Alle vragen vind je in dit notebook. Lever dit notebook weer in via Brightspace voordat de tijd voorbij is! 
Als je extra tijd hebt, wordt je inlevering als 'late' bestempeld, dat is geen probleem.

Veel succes!

### Vraag 1
20 punten (2 punten per onderdeel)

Hieronder vind je een stukje code. Geef voor ieder van deze concepten of programmeertechnieken aan of ze toegepast zijn in het stukje code. Als ze worden toegepast geef je ook aan op welke plek dat is, één keer aangeven is voldoende!

Geef antwoord door het commentaar bovenaan de code aan te passen. Geef aan waar het in de code wordt toegepast door commentaar in de code te plaatsen.

1. Wordt er een closure gebruikt? Zo ja, waar?
2. Wordt functiecompositie gebruikt? Zo ja, waar?
3. Wordt partial function application gebruikt? Zo ja, waar?
4. Wordt er recursie gebruikt? Zo ja, waar?
5. Wordt pattern matching gebruikt? Zo ja, waar?
6. Wordt er een pure functie gebruikt? Zo ja, waar?
7. Wordt er een onpure (niet puur) functie gebruikt? Zo ja, waar?
8. Wordt er een tuple gebruikt? Zo ja, waar?
9. Wordt er een discriminated union (enum) gebruikt? Zo ja, waar?
10. Wordt er een lambda functie gebruikt? Zo ja, waar?

In [ ]:
// 1. Closure: wel
// 2. Functiecompositie: wel
// 3. Partial function application: niet
// 4. Recursie: wel
// 5. Pattern matching: wel
// 6. Pure functie: wel
// 7. Onpure functie: wel
// 8. Tuple: wel
// 9. Discriminated union (enum): wel
// 10. Lambda: wel

// Dit is een immutable binary search tree, hoe cool 😎

type TreeNode = // Discriminated union (enum)
    | Empty 
    | Leaf of int  
    | Left of int * TreeNode //Tuple
    | Right of int * TreeNode 
    | LeftAndRight of int * TreeNode * TreeNode 

let rec insert node value = // Pure functie
    match node with  // patern matching
        | Empty -> Leaf value
        | Leaf current when value <= current -> Left (current, Leaf value) 
        | Leaf current -> Right (current, Leaf value)
        | Left (current, leftChild) when value <= current -> Left (current, insert leftChild value)
        | Left (current, leftChild) -> LeftAndRight (current, leftChild, Leaf value)
        | Right (current, rightChild) when value <= current -> LeftAndRight (current, Leaf value, rightChild)
        | Right (current, rightChild) -> Right (current, insert rightChild value)
        | LeftAndRight (current, leftChild, rightChild) when value <= current -> LeftAndRight (current, insert leftChild value, rightChild)
        | LeftAndRight (current, leftChild, rightChild) -> LeftAndRight (current, leftChild, insert rightChild value)

let rec contains node value = // Pure functie
    match node with // patern matching
        | Empty -> false
        | Leaf current -> value = current
        | Left (current, leftChild) when value < current -> contains leftChild value
        | Left (current, _) when value = current -> true
        | Left _ -> false
        | Right (current, rightChild) when value > current -> contains rightChild value
        | Right (current, _) when value = current -> true
        | Right _ -> false
        | LeftAndRight (current, leftChild, _) when value < current -> contains leftChild value
        | LeftAndRight (current, _, _) when value = current -> true
        | LeftAndRight (current, _, rightChild) -> contains rightChild value

let rec buildTree strategy =  // Pure functie
    match strategy () with
        | Some value -> insert (buildTree strategy) value // recursie
        | None -> Empty

let makeTestNumberStrategy () = //Onpure functie
    let mutable nums = [10; 1; 11; 3; 2; 12]
    fun () -> // closure nums // Lambda
        match nums with
            | [] -> None
            | head :: tail -> 
                nums <- tail
                Some head

let makeRandomNumberStrategy amount () = //Onpure functie
    let rnd = (new System.Random())
    let mutable index = 0
    fun () -> // closure rnd, index,  Lambda
        if index < amount then
            index <- index + 1
            Some (rnd.Next(0, 100))
        else None


let buildTestTree = makeTestNumberStrategy >> buildTree // function composition
let buildRandomTree = makeRandomNumberStrategy 10 >> buildTree // function composition

let testTree = buildTestTree ()
let randomTree = buildRandomTree ()

printfn "%O" (testTree)
printfn "%O" (randomTree)

printfn "test tree contains 11: %b" (contains testTree 11)
printfn "test tree contains 9: %b" (contains testTree 9)

printfn "random tree contains 42: %b" (contains randomTree 42)

Left (12, LeftAndRight (2, Leaf 1, Right (3, Left (11, Leaf 10))))
LeftAndRight
  (84,
   LeftAndRight
     (13, Right (2, Leaf 8),
      LeftAndRight (56, Left (43, Left (21, Leaf 20)), Leaf 70)), Leaf 94)
test tree contains 11: true
test tree contains 9: false
random tree contains 42: false


### Vraag 2.a
10 punten

Schrijf een pure recursieve functie die een lijst van tuples van twee integer opties afgaat en de som per tuple berekent. De som kan alleen berekent worden als beide opties `Some` zijn. Als minimaal één van de twee `None` is wordt het tuple genegeerd. Gebruik hierbij pattern matching.

Voorbeeld:
```F#
sumOptions [(Some 1, Some 2); (Some 1, None); (None, Some 3); (Some 5, Some 3); (None, None); (Some 0, Some 0)] // geeft [3; 8; 0]
```

In [ ]:
let rec sumOptions list = 
    match (list) with
        | [] -> [];
        | (Some x, Some y) :: tail -> x + y :: sumOptions tail
        | _ :: tail -> sumOptions tail

sumOptions [(Some 1, Some 2); (Some 1, None); (None, Some 3); (Some 5, Some 3); (None, None); (Some 0, Some 0)]

index,value
0,3
1,8
2,0


### Vraag 2.b
10 punten

Schrijf een pure **tail** recursieve functie die alle Errors uit een lijst met Results haalt en achter elkaar zet. De waarde in de Ok kan worden genegeerd. (Het maakt niet uit hoe de uiteindelijk string er precies uit ziet, als alle error berichten er maar in staan.)

Voorbeeld:
```F#
getAllErrors [Error "something went wrong"; Ok 5; Ok 3; Error "uh oh..."; Error "oopsie"] "errors:\n" 
// geeft: 
// "errors:
//  something went wrong
//  uh oh...
//  oopsie"
```

In [ ]:
// Ik dacht eerst dat er een lijst met errors teruggegeven moest worden, dus daarom bestaat deze functie.
let rec getAllErrorsRec list result =
    match list with 
        | [] -> result
        | Error value :: tail -> getAllErrorsRec tail (value :: result)
        | _ :: tail -> getAllErrorsRec tail result 

let getAllErrors list = getAllErrorsRec list [] 

// Daarna las ik nog eens (deze keer goed) en zag dat er een string moest worden teruggegeven, 
// dus is deze functie gemaakt. Ik heb getAllErrorsRec niet verwijderd omdat ik dat zonde vond.
let rec getAllErrorsString list result =
    match list with 
        | [] -> result
        | Error value :: tail -> getAllErrorsString tail (result + value + "\n")
        | _ :: tail -> getAllErrorsString tail result

getAllErrors [Error "something went wrong"; Ok 5; Ok 3; Error "uh oh..."; Error "oopsie"]  
getAllErrorsString [Error "something went wrong"; Ok 5; Ok 3; Error "uh oh..."; Error "oopsie"]  "errors:\n"

//"errors:\n" 
// geeft: 
// "errors:
//  something went wrong
//  uh oh...
//  oopsie"

errors:
something went wrong
uh oh...
oopsie


### Vraag 3 (bestaat uit twee delen)
20 (= 10 + 10) punten

Gegeven is een discriminated union `ContactInfo` waarmee een telefoonnummer en/of een emailadres kan worden vertegenwoordigd. Ook zijn er de functies `removePhone` en `setDefaultEmail` gegeven. `removePhone` verwijdert het telefoonnummer en `setDefaultEmail` voegt er een default emailadres aan toe. `setDefaultEmail` krijgt daarvoor een strategie (functie met signature `() -> string`) mee die nieuwe default adressen genereert.

Hier zie je voorbeelden van hoe de functies gebruikt kunnen worden:
```F#
removePhone (PhoneOnly "1234567890") // geeft Neither
removePhone (EmailAndPhone ("1234567890", "rma.vanderheiden@avans.nl")) // geeft EmailOnly "rma.vanderheiden@avans.nl"

let constantDefault () = "default@example.com"

setDefaultEmail constantDefault (EmailOnly "rma.vanderheiden@avans.nl") // geeft EmailOnly "rma.vanderheiden@avans.nl"
setDefaultEmail constantDefault (PhoneOnly "1234567890") // geeft PhoneAndEmail ("1234567890", "default@example.com")
```
1. Schrijf een functie `makeEmailFactory` die een default emailgenerator bouwt. `makeEmailFactory` geeft een functie terug met signature `() -> string` en iedere keer dat deze functie wordt aangeroepen wordt er een nieuw opvolgend defaultadres gegenereerd: "me0@example.com", "me1@example.com", "me2@example.com", et cetera. Je moet hierbij waarschijnlijk een mutable variabele gebruiken.
```F#
let emailFactory = makeEmailFactory ()

emailFactory () // geeft "me0@example.com"
emailFactory () // geeft "me1@example.com"
emailFactory () // geeft "me2@example.com"
```
2. Als vraag 3.1 niet helemaal correct werkt is de code voor deze vraag nog steeds goed te schrijven, maar de voorbeeldresultaten komen dan waarschijnlijk niet overeen. Schrijf een functie `getEmail` met signature `ContactInfo -> ContactInfo` die altijd een EmailOnly teruggeeft. Het emailadres dat erin zit moet het specifieke adres zijn als dat bestaat en anders gevuld zijn met een default adres. Schrijf deze functie op basis van `setDefaultEmail` en `removePhone` met functiecompositie. Gebruik hierbij partial application en de functie `makeEmailFactory`. (Dit gaat dus allemaal in 1 of 2 regels!)
```F#
getEmail Neither // EmailOnly "me0@example.com"
getEmail (PhoneOnly "1234567890") // EmailOnly "me1@example.com"
getEmail (EmailOnly "dj.koeze@avans.nl") // EmailOnly "dj.koeze@avans.nl"
getEmail (PhoneAndEmail ("1234567890", "dj.koeze@avans.nl")) // EmailOnly "dj.koeze@avans.nl"
```

In [ ]:
type ContactInfo =
    | Neither
    | PhoneOnly of string
    | EmailOnly of string
    | PhoneAndEmail of string * string

let removePhone info =
    match info with
        | Neither -> Neither // niets om te verwijderen
        | PhoneOnly _ -> Neither // niets blijft over met het telefoonnummer verwijderd
        | EmailOnly _ -> info // alleen een email, hoeft niets aan te veranderen
        | PhoneAndEmail (_, email) -> EmailOnly email // alleen de email blijft over

let setDefaultEmail makeDefaultEmail info =
    match info with
        | Neither -> EmailOnly (makeDefaultEmail ()) // zet alleen een defaultemail
        | PhoneOnly phone -> PhoneAndEmail (phone, (makeDefaultEmail ())) // voeg een defaultemail toe
        | EmailOnly _ -> info // hoeft niets te veranderen
        | PhoneAndEmail _ -> info // hoeft niets te veranderen

        
// schrijf hier je uitwerking
let makeEmailFactory =
    let mutable number = 0
    fun () -> 
        let x = $"me%d{number}@example.com" 
        number <- number + 1
        x    


let getEmail contactInfo =
    let compositeFunction =  setDefaultEmail makeEmailFactory  >> removePhone 
    compositeFunction contactInfo
    

printfn "%O" (getEmail Neither) // EmailOnly "me0@example.com"
printfn "%O" (getEmail (PhoneOnly "1234567890")) // EmailOnly "me1@example.com"
printfn "%O" (getEmail (EmailOnly "dj.koeze@avans.nl")) // EmailOnly "dj.koeze@avans.nl"
printfn "%O" (getEmail (PhoneAndEmail ("1234567890", "dj.koeze@avans.nl"))) // EmailOnly "dj.koeze@avans.nl"

EmailOnly "me0@example.com"
EmailOnly "me1@example.com"
EmailOnly "dj.koeze@avans.nl"
EmailOnly "dj.koeze@avans.nl"


### Vraag 4 (RxAvans)
30 punten

We willen twee nieuwe operatoren toevoegen aan RxAvans: `duplicate` en `sum`. Jij gaat deze twee implementeren en je kan hiermee 15 punten per operator verdienen. Hieronder staan ze beschreven en vind je code om je uitwerking uit te proberen. Je kan ook zelf testcode schrijven.

##### Duplicate
De duplicate operator stuurt de waarde die hij binnenkrijgt meerdere keren door. Het aantal keer dat het doorgestuurd wordt is een parameter. 

Voorbeeld (dubbelklik voor copy): 
```F#
let producer observer = for i in 1..3 do observer i

let doubled = duplicate 2 producer

doubled (printfn "received: %A")
// received: 1
// received: 1
// received: 2
// received: 2
// received: 3
// received: 3
```

##### Sum

De som werkt alleen op een pipeline waar getallen doorheen gaan. Deze operator houdt bij wat de som van alle getallen die tot nu toe voorbij gekomen is en stuurt deze som iedere keer door. Je moet hierbij waarschijnlijk een mutable variabele gebruiken.

Voorbeeld:
```F#
let produceOnes = for i in 1..5 observer 1 // let op: geeft iedere keer 1
let produceOneToFive observer = for i in 1..5 do observer i // let op: geeft 1, dan 2, dan 3, dan 4 en dan 5

let summedOnes = sum produceOnes
let summedOneToFive = sum produceOneToFive

summedOnes (printfn "received: %A")
// received: 1
// received: 2
// received: 3
// received: 4
// received: 5

summedOneToFive (printfn "received: %A")
// received: 1
// received: 3
// received: 6
// received: 10
// received: 15
```

In [ ]:
let log previous =
    fun next ->
        previous (fun data -> 
            printfn "logging: %i" data
            next data
        )

let tap func previous = 
    fun next ->
        previous (fun data ->
            func data
            next data
        )

let filter predicate previous =
    fun next -> 
        previous (fun data -> 
            if predicate data then next data
        )

let map mapper previous =
    fun next ->
        previous (fun data ->
            next (mapper data)
        )



let skip amount previous =
    let mutable count = 0
    fun next ->
        previous (fun data -> 
            if (count >= amount) then next data
            count <- count + 1
        )

let take amount previous =
    let mutable count = 0
    fun next ->
        previous (fun data ->
            if count < amount then next data
            count <- count + 1
        )    


// schrijf hier je uitwerking
let duplicate amount previous =
    fun next ->
        previous (fun data ->
            for i = 1 to amount do
            next data
        )

let sum previous =
    let mutable sum = 0
    fun next ->
        previous (fun data ->
            sum <- sum + data
            next sum 
        
        )


let producer observer = for i in 1..6 do observer i

let tripled = duplicate 3 producer
let summed = sum tripled

producer (printfn "you see me once: %i")
// you see me once: 1
// you see me once: 2
// you see me once: 3
// you see me once: 4
// you see me once: 5
// you see me once: 6

tripled (printfn "you see me three times: %i") 
// you see me three times: 1
// you see me three times: 1
// you see me three times: 1
// you see me three times: 2
// you see me three times: 2
// you see me three times: 2
// you see me three times: 3
// you see me three times: 3
// you see me three times: 3
// you see me three times: 4
// you see me three times: 4
// you see me three times: 4
// you see me three times: 5
// you see me three times: 5
// you see me three times: 5
// you see me three times: 6
// you see me three times: 6
// you see me three times: 6

summed (printfn "sum up to now: %i")
// sum up to now: 1
// sum up to now: 2
// sum up to now: 3
// sum up to now: 5
// sum up to now: 7
// sum up to now: 9
// sum up to now: 12
// sum up to now: 15
// sum up to now: 18
// sum up to now: 22
// sum up to now: 26
// sum up to now: 30
// sum up to now: 35
// sum up to now: 40
// sum up to now: 45
// sum up to now: 51
// sum up to now: 57
// sum up to now: 63

// geschreven als een pipeline:
(producer |> duplicate 3 |> sum) (printfn "sum from pipeline %i")

you see me once: 1
you see me once: 2
you see me once: 3
you see me once: 4
you see me once: 5
you see me once: 6
you see me three times: 1
you see me three times: 1
you see me three times: 1
you see me three times: 2
you see me three times: 2
you see me three times: 2
you see me three times: 3
you see me three times: 3
you see me three times: 3
you see me three times: 4
you see me three times: 4
you see me three times: 4
you see me three times: 5
you see me three times: 5
you see me three times: 5
you see me three times: 6
you see me three times: 6
you see me three times: 6
sum up to now: 1
sum up to now: 2
sum up to now: 3
sum up to now: 5
sum up to now: 7
sum up to now: 9
sum up to now: 12
sum up to now: 15
sum up to now: 18
sum up to now: 22
sum up to now: 26
sum up to now: 30
sum up to now: 35
sum up to now: 40
sum up to now: 45
sum up to now: 51
sum up to now: 57
sum up to now: 63
sum from pipeline 1
sum from pipeline 2
sum from pipeline 3
sum from pipeline 5
sum from pipeline 7
